<a href="https://colab.research.google.com/github/AbhishekAjoy/ML-DL/blob/master/BertEvalTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
fpath = "/content/SICK_train.txt"
import pandas as pd
df = pd.read_csv(fpath, sep='\t')

In [3]:
len(df["sentence_B"])

4500

In [4]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 2.3MB 9.3MB/s 
     |████████████████████████████████| 1.2MB 28.3MB/s 
     |████████████████████████████████| 3.3MB 38.0MB/s 
     |████████████████████████████████| 901kB 37.7MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=e0065dd5f87953f7b443658546c17a5eff91619db43119b5181832da73a65a40
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import plotly.graph_objects as go

In [7]:
model_name = "bert-base-nli-mean-tokens"
model  = SentenceTransformer(model_name)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
no_of_sentences = 4500

In [9]:
scores = []
for i in range(0,no_of_sentences): 
  sentences = [df["sentence_A"][i],df["sentence_B"][i]]
  sentence_vecs = model.encode(sentences)
  cosine =cosine_similarity(
      [sentence_vecs[0]],
      [sentence_vecs[1]]
  )
  scores.append(cosine[0][0] * 5)

In [19]:

rms = mean_squared_error(df["relatedness_score"][:no_of_sentences], scores, squared=False)
print('Root Mean Squared Error: %.3f' % rms)

Root Mean Squared Error: 0.962


In [18]:
corr, _ = pearsonr(df["relatedness_score"], scores)
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.731


In [29]:
X = ['sentence pair 1','sentence pair 2','sentence pair 3','sentence pair 4','sentence pair 5']
fig = go.Figure(
    data=[
    go.Bar(name='Actual Score', x = X,y=df['relatedness_score'][:5]),
    go.Bar(name='Predicted Score', x = X,y=scores[:5])],
    layout={
        'yaxis': {'title': 'Similarity Score(max: 5)'},
        'xaxis': {'title': 'Sentence Pairs'}
    }
)
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

In [14]:
scores[:10]

[4.586551785469055,
 2.0277002453804016,
 4.810570180416107,
 3.7734967470169067,
 3.6788254976272583,
 3.4170395135879517,
 3.7192708253860474,
 2.986401617527008,
 1.5523238480091095,
 2.7433401346206665]

In [15]:
df['relatedness_score'][:10]

0    4.5
1    3.2
2    4.7
3    3.4
4    3.7
5    4.0
6    3.5
7    3.2
8    2.8
9    3.7
Name: relatedness_score, dtype: float64